# wikipedia hyperlink network visualization

data: https://snap.stanford.edu/data/wiki-topcats.html

source: Jure Leskovec, Stanford University

> This is a web graph of Wikipedia hyperlinks collected in September 2011. The network was constructed by first taking the largest strongly connected component of Wikipedia, then restricting to pages in the top set of categories (those with at least 100 pages), and finally taking the largest strongly connected component of the restricted graph.
 
The goal is to utilise __PySpark__ to handle the data and to visualize and understand the network


## the data

the data rests in 3 text files of each ±40MB
- categories
    - a line looks like: Category:Category_name; (series of numbers)
    - Category:Buprestoidea; 301 302 303
    - the numbers presumably represent the pages in the category 
- page names
    - a line looks like: number Page_name
    - 0 Chiasmal syndrome
    - this represents the mappin of number to name
- reduced
    - a line consists of 2 tab separated numbers
    - 52	401135
    - the numbers presumably represent a hyperlink so __edges in the graph__


### setup

- count number of pages ( lines of page doc)
- show all page names in a category 

In [ ]:
from pyspark import SparkContext
#import pyspark.sql.functions as F

In [ ]:
try:
    sc.stop()
    sc = SparkContext('local')
except NameError:
    sc = SparkContext('local')

In [ ]:
names = sc.textFile("../data/wiki-topcats-page-names.txt")
edges = sc.textFile("../data/wiki-topcats-reduced.txt")

In [ ]:
print(names.count(),edges.count())

### some helper methods

In [ ]:
def getNameByNumber(num):
    num_str = str(num)+' '
    name_filter = names.filter(lambda x: num_str in x[:len(str(num))+1])\
        .map(lambda x: x[len(str(num))+1:])
    name = name_filter.collect()
    return name[0]
    
def getLinksByNumber(num):
    num_str = str(num)+'\t'
    edge_filter = edges.filter(lambda x: num_str in x[:len(str(num))+1])\
        .map(lambda x: x[len(str(num))+1:])
    links = edge_filter.collect()
    return links
def getNextNamesByNumber(num):
    next_nums = getLinksByNumber(num)
    names=[]
    for n in next_nums:
        names.append(getNameByNumber(n))
    return names
def hasLinks(num):
    # check getAllPagesWithLinks list in stead of whole doc
    num_str = str(num)+'\t'
    edge_filter = edges.filter(lambda x: num_str in x[:len(str(num))+1])
    if(edge_filter.count()==0):
        return False
    return True

def getAllPagesWithLinks():
    return


In [ ]:
number = 8484
print(getNameByNumber(number))
if(hasLinks(number)):
    print(getNextNamesByNumber(number))
else:
    print("no links")

## display small example network

- start with a number
- get links
- draw edge from name to next names

In [ ]:
num_in = 62
name_in = getNameByNumber(num_in)
next_nums = getLinksByNumber(num_in)

import networkx as nx
import matplotlib.pyplot as plt

G = nx.Graph()

for num in next_nums:  
    G.add_edge(name_in,getNameByNumber(num))

nx.draw(G, with_labels=True)
plt.show()



In [ ]:

print(names.take(10))

### create network matrix, visualize & check sparsity

1791488 nodes

2645248 edges




In [ ]:
import matplotlib.pylab as plt
import numpy as np
import scipy.sparse as sp

B = sp.lil_matrix((1791489,1791489),dtype=np.int8)

sample =edges.take(10)
s_edges= sc.parallelize(sample)

split_edges= edges.map(lambda x: x.split('\t')).collect()

In [ ]:

for id,x in enumerate(split_edges):
    B[int(x[0]),int(x[1])]=1
    if(id%100000==0):
        print(id)
print('matrix loaded')

plotting the matrix is a large and slow operation (+300MB)

so the code is commented out and here is a picture representation

<img src="matrix.png" alt="alt text" width="400"/>



In [20]:
# plot the matrix
#fig = plt.spy(B,markersize=0.01,marker=".")
#fig.figure.savefig('matrix.png',dpi=500)